# Backpropagation Practice

Implement a 3 input, 4 node hidden-layer, 1 output node Multilayer Perceptron on the following dataset:

| x1 | x2 | x3 | y |
|----|----|----|---|
| 0  | 0  | 1  | 0 |
| 0  | 1  | 1  | 1 |
| 1  | 0  | 1  | 1 |
| 0  | 1  | 0  | 1 |
| 1  | 0  | 0  | 1 |
| 1  | 1  | 1  | 0 |
| 0  | 0  | 0  | 0 |

If you look at the data you'll notice that the first two columns behave like an XOR gate while the last column is mostly just noise. Remember that creating an XOR gate was what the perceptron was criticized for not being able to learn. 

In [1]:
##### Your Code Here #####
import numpy as np

X = np.array([[0.,0.,1.],
               [0.,1.,1.],
               [1.,0.,1.],
               [0.,1.,0.],
               [1.,0.,0.],
               [1.,1.,1.],
               [0.,0.,0.]])

y = np.array([[0.],
              [1.],
              [1.],
              [1.],
              [1.],
              [0.],
              [0.]])

In [2]:
class Neural_Network(object):
    def __init__(self):
        self.inputs = 3
        self.hiddenNodes = 4
        self.outputNodes = 1

    # Initlize Weights
        self.L1_weights = np.random.randn(self.inputs, self.hiddenNodes) # (3x4)
        self.L2_weights = np.random.randn(self.hiddenNodes, self.outputNodes) # (4x1)

    def feed_forward(self, X):
        # Weighted sum between inputs and hidden layer
        self.hidden_sum = np.dot(X, self.L1_weights)
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        # Weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
        # final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        return self.activated_output

    def sigmoid(self, s):
        return 1/(1+np.exp(-s))

General Output:

In [3]:
# General output
NN = Neural_Network()
output = NN.feed_forward(X)
print("Output:\n", output)

Output:
 [[0.27508871]
 [0.37592531]
 [0.39622093]
 [0.35895832]
 [0.37572934]
 [0.46772734]
 [0.25427298]]


Error with General Output:

In [4]:
# expected - predicted
print("Error:\n",y - output)

Error:
 [[-0.27508871]
 [ 0.62407469]
 [ 0.60377907]
 [ 0.64104168]
 [ 0.62427066]
 [-0.46772734]
 [-0.25427298]]


## Backpropagation (Assigning blame for bad predictions and delivering justice - repeatedly and a little bit at a time)

What in our model could be causing our predictions to suck so bad? 

Well, we know that our inputs (X) and outputs (y) are correct, if they weren't then we would have bigger problems than understanding backpropagation.

We also know that our activation function (sigmoid) is working correctly. It can't be blamed because it just does whatever we tell it to and transforms the data in a known way.

So what are the potential culprits for these terrible predictions? The **weights** of our model. Here's the problem though. I have weights that exist in both layers of my model. How do I know if the weights in the first layer are to blame, or the second layer, or both? 

Lets investigate. And see if we can just eyeball what should be updated.

In [5]:
class Neural_Network(object):
    def __init__(self):
        self.inputs = 3
        self.hiddenNodes = 4
        self.outputNodes = 1

    # Initlize Weights
        self.L1_weights = np.random.randn(self.inputs, self.hiddenNodes) # (3x4)
        self.L2_weights = np.random.randn(self.hiddenNodes, self.outputNodes) # (4x1)

    def feed_forward(self, X):
        # Weighted sum between inputs and hidden layer
        self.hidden_sum = np.dot(X, self.L1_weights)
        # Activations of weighted sum
        self.activated_hidden = self.sigmoid(self.hidden_sum)
        # Weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
        # final activation of output
        self.activated_output = self.sigmoid(self.output_sum)
        return self.activated_output

    def sigmoid(self, s):
        return 1/(1+np.exp(-s))
   
    # sigmoidPrime: derivative of sigmoid function 
    def sigmoidPrime(self, s):
        return s * (1 - s)
    
    # backpropagation function
    # backward propagate through the network
    def backward(self, X, y, output):
        # error in output for every prediction
        self.output_error = y - output
        # applying derivative of sigmoid to error
        # delta: how much I want to change all my weights 
        # given my error multiplied by derivative sigmoid of output
        self.output_delta = self.output_error * self.sigmoidPrime(output)
        
        # z2 error: how much our hidden layer weights (2nd to last layer) 
        # contributed to output error
        self.z2_error = self.output_delta.dot(self.L2_weights.T)
        # applying derivative of sigmoid to z2 error
        # z2_delta: how much weights in this layer will change based on error calculated
        self.z2_delta = self.z2_error * self.sigmoidPrime(self.activated_hidden)
        
        # Here: updating the 2 sets of weights 
        # (between input & hidden AND between hidden & output)
        # L1_weights: adjusting first set (input --> hidden) weights
        self.L1_weights += X.T.dot(self.z2_delta)
        # L2_weights: adjusting second set (hidden --> output) weights
        self.L2_weights += self.activated_hidden.T.dot(self.output_delta)
    
    # Telling Network to run certain # of times
    def train(self, X, y):
        output = self.feed_forward(X)
        self.backward(X, y, output)

In [6]:
# Instantiating Neural_Network object
# Running it 1000 times
# And Printing out every 50th Epoch
NN = Neural_Network()
for i in range(1000): # trains the NN 1,000 times
    if i+1 in [1,2,3,4,5] or (i+1) % 50 == 0:
        print('+---------- EPOCH', i+1, '-----------+')
        print("Input: \n", X) 
        print("Actual Output: \n", y)  
        print("Predicted Output: \n" + str(NN.feed_forward(X))) 
        print("Loss: \n" + str(np.mean(np.square(y - NN.feed_forward(X))))) # mean sum squared loss
        print("\n")
    NN.train(X, y)

+---------- EPOCH 1 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[0.41889582]
 [0.50553825]
 [0.48411313]
 [0.40945109]
 [0.36914483]
 [0.54337666]
 [0.30071468]]
Loss: 
0.2597884525694637


+---------- EPOCH 2 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[0.45912393]
 [0.54887971]
 [0.53192677]
 [0.45947723]
 [0.42397018]
 [0.59050771]
 [0.34195413]]
Loss: 
0.2461434355707411


+---------- EPOCH 3 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[0.48489247]
 [0.57857705]
 [0.56480307]
 [0.49656253]
 [0.46518465]
 [0.62350879]
 [0.37153598]]
Loss: 
0.2383419028813026


+

In [7]:
##### Your Code Here #####
import numpy as np

X = np.array([[0.,0.,1.],
               [0.,1.,1.],
               [1.,0.,1.],
               [0.,1.,0.],
               [1.,0.,0.],
               [1.,1.,1.],
               [0.,0.,0.]])

y = np.array([[0.],
              [1.],
              [1.],
              [1.],
              [1.],
              [0.],
              [0.]])

In [8]:
class Neural_Network_ReLU(object):
    def __init__(self):
        self.inputs = 3
        self.hiddenNodes = 4
        self.outputNodes = 1

    # Initlize Weights
        self.L1_weights = np.random.randn(self.inputs, self.hiddenNodes) # (3x4)
        self.L2_weights = np.random.randn(self.hiddenNodes, self.outputNodes) # (4x1)

    def feed_forward(self, X):
        # Weighted sum between inputs and hidden layer
        self.hidden_sum = np.dot(X, self.L1_weights)
        # Activations of weighted sum
        self.activated_hidden = self.ReLU(self.hidden_sum)
        # Weighted sum between hidden and output
        self.output_sum = np.dot(self.activated_hidden, self.L2_weights)
        # final activation of output
        self.activated_output = self.ReLU(self.output_sum)
        return self.activated_output

    def ReLU(self, s):
        return s * (s > 0)
  
    def ReLUPrime(self, s):
        s_copy = s
        s_copy[s_copy<=0] = 0
        s_copy[s_copy>0] = 1
        return s_copy
    
    # backpropagation function
    # backward propagate through the network
    def backward(self, X, y, output):
        # error in output for every prediction
        self.output_error = y - output
        # applying derivative of ReLU to error
        # delta: how much I want to change all my weights 
        # given my error multiplied by derivative ReLU of output
        self.output_delta = self.output_error * self.ReLUPrime(output)
        
        # z2 error: how much our hidden layer weights (2nd to last layer) 
        # contributed to output error
        self.z2_error = self.output_delta.dot(self.L2_weights.T)
        # applying derivative of ReLU to z2 error
        # z2_delta: how much weights in this layer will change based on error calculated
        self.z2_delta = self.z2_error * self.ReLUPrime(self.activated_hidden)
        
        # Here: updating the 2 sets of weights 
        # (between input & hidden AND between hidden & output)
        # L1_weights: adjusting first set (input --> hidden) weights
        self.L1_weights += X.T.dot(self.z2_delta)
        # L2_weights: adjusting second set (hidden --> output) weights
        self.L2_weights += self.activated_hidden.T.dot(self.output_delta)
    
    # Telling Network to run certain # of times
    def train(self, X, y):
        output = self.feed_forward(X)
        self.backward(X, y, output)

Somehow this never converged using ReLU function!!! Don't know why... 

In [9]:
NN = Neural_Network_ReLU()
for i in range(10000): # trains the NN 10,000 times
    if i+1 in [1,2,3,4,5] or (i+1) % 1000 == 0:
        print('+---------- EPOCH', i+1, '-----------+')
        print("Input: \n", X) 
        print("Actual Output: \n", y)  
        print("Predicted Output: \n" + str(NN.feed_forward(X))) 
        print("Loss: \n" + str(np.mean(np.square(y - NN.feed_forward(X))))) # mean sum squared loss
        print("\n")
    NN.train(X, y)

+---------- EPOCH 1 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [ 0.]]
Loss: 
0.5714285714285714


+---------- EPOCH 2 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [ 0.]]
Loss: 
0.5714285714285714


+---------- EPOCH 3 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.]
 [1.]
 [1.]
 [1.]
 [1.]
 [0.]
 [0.]]
Predicted Output: 
[[-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [-0.]
 [ 0.]]
Loss: 
0.5714285714285714


+---------- EPOCH 4 -----------+
Input: 
 [[0. 0. 1.]
 [0. 1. 1.]
 [1. 0. 1.]
 [0. 1. 0.]
 [1. 0. 0.]
 [1. 1. 1.]
 [0. 0. 0.]]
Actual Output: 
 [[0.

## Try building/training a more complex MLP on a bigger dataset.

Use the [MNIST dataset](http://yann.lecun.com/exdb/mnist/) to build the cannonical handwriting digit recognizer and see what kind of accuracy you can achieve. 

If you need inspiration, the internet is chalk-full of tutorials, but I want you to see how far you can get on your own first. I've linked to the original MNIST dataset above but it will probably be easier to download data through a neural network library. If you reference outside resources make sure you understand every line of code that you're using from other sources, and share with your fellow students helpful resources that you find.


In [10]:
from mlxtend.data import mnist_data
X, y = mnist_data()

In [11]:
X.shape, y.shape

((5000, 784), (5000,))

In [12]:
X[0]

array([  0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,  51., 159., 253., 159.,  50.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,   0.,
         0.,   0.,   0.,   0.,   0.,   0.,   0.,   

In [14]:
dataset = np.concatenate((X, y[:, None]), 1)
np.savetxt("minst.csv", dataset, delimiter=",")

In [17]:
# Backprop on the Seeds Dataset
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp

# Load a CSV file
def load_csv(filename):
	dataset = list()
	with open(filename, 'r') as file:
		csv_reader = reader(file)
		for row in csv_reader:
			if not row:
				continue
			dataset.append(row)
	return dataset

# Convert string column to float
def str_column_to_float(dataset, column):
	for row in dataset:
		row[column] = float(row[column].strip())
        
# Convert string column to integer
def str_column_to_int(dataset, column):
	class_values = [row[column] for row in dataset]
	unique = set(class_values)
	lookup = dict()
	for i, value in enumerate(unique):
		lookup[value] = i
	for row in dataset:
		row[column] = lookup[row[column]]
	return lookup

# Find the min and max values for each column
def dataset_minmax(dataset):
	minmax = list()
	stats = [[min(column), max(column)] for column in zip(*dataset)]
	return stats

# Rescale dataset columns to the range 0-1
def normalize_dataset(dataset, minmax):
	for row in dataset:
		for i in range(len(row)-1):
				if (minmax[i][1] != minmax[i][0]):
						row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])
                        
# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
	dataset_split = list()
	dataset_copy = list(dataset)
	fold_size = int(len(dataset) / n_folds)
	for i in range(n_folds):
		fold = list()
		while len(fold) < fold_size:
			index = randrange(len(dataset_copy))
			fold.append(dataset_copy.pop(index))
		dataset_split.append(fold)
	return dataset_split

# Calculate accuracy percentage
def accuracy_metric(actual, predicted):
	correct = 0
	for i in range(len(actual)):
		if actual[i] == predicted[i]:
			correct += 1
	return correct / float(len(actual)) * 100.0

# Evaluate an algorithm using a cross validation split
def evaluate_algorithm(dataset, algorithm, n_folds, *args):
	folds = cross_validation_split(dataset, n_folds)
	scores = list()
	for fold in folds:
		train_set = list(folds)
		train_set.remove(fold)
		train_set = sum(train_set, [])
		test_set = list()
		for row in fold:
			row_copy = list(row)
			test_set.append(row_copy)
			row_copy[-1] = None
		predicted = algorithm(train_set, test_set, *args)
		actual = [row[-1] for row in fold]
		accuracy = accuracy_metric(actual, predicted)
		scores.append(accuracy)
	return scores

# Calculate neuron activation for an input
def activate(weights, inputs):
	activation = weights[-1]
	for i in range(len(weights)-1):
		activation += weights[i] * inputs[i]
	return activation

# Transfer neuron activation
def transfer(activation):
	return 1.0 / (1.0 + exp(-activation))

# Forward propagate input to a network output
def forward_propagate(network, row):
	inputs = row
	for layer in network:
		new_inputs = []
		for neuron in layer:
			activation = activate(neuron['weights'], inputs)
			neuron['output'] = transfer(activation)
			new_inputs.append(neuron['output'])
		inputs = new_inputs
	return inputs

# Calculate the derivative of an neuron output
def transfer_derivative(output):
	return output * (1.0 - output)

# Backpropagate error and store in neurons
def backward_propagate_error(network, expected):
	for i in reversed(range(len(network))):
		layer = network[i]
		errors = list()
		if i != len(network)-1:
			for j in range(len(layer)):
				error = 0.0
				for neuron in network[i + 1]:
					error += (neuron['weights'][j] * neuron['delta'])
				errors.append(error)
		else:
			for j in range(len(layer)):
				neuron = layer[j]
				errors.append(expected[j] - neuron['output'])
		for j in range(len(layer)):
			neuron = layer[j]
			neuron['delta'] = errors[j] * transfer_derivative(neuron['output'])

# Update network weights with error
def update_weights(network, row, l_rate):
	for i in range(len(network)):
		inputs = row[:-1]
		if i != 0:
			inputs = [neuron['output'] for neuron in network[i - 1]]
		for neuron in network[i]:
			for j in range(len(inputs)):
				neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
			neuron['weights'][-1] += l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs):
	for epoch in range(n_epoch):
		for row in train:
			outputs = forward_propagate(network, row)
			expected = [0 for i in range(n_outputs)]
			expected[row[-1]] = 1
			backward_propagate_error(network, expected)
			update_weights(network, row, l_rate)

# Initialize a network
def initialize_network(n_inputs, n_hidden, n_outputs):
	network = list()
	hidden_layer = [{'weights':[random() for i in range(n_inputs + 1)]} for i in range(n_hidden)]
	network.append(hidden_layer)
	output_layer = [{'weights':[random() for i in range(n_hidden + 1)]} for i in range(n_outputs)]
	network.append(output_layer)
	return network

# Make a prediction with a network
def predict(network, row):
	outputs = forward_propagate(network, row)
	return outputs.index(max(outputs))

# Backpropagation Algorithm With Stochastic Gradient Descent
def back_propagation(train, test, l_rate, n_epoch, n_hidden):
	n_inputs = len(train[0]) - 1
	n_outputs = len(set([row[-1] for row in train]))
	network = initialize_network(n_inputs, n_hidden, n_outputs)
	train_network(network, train, l_rate, n_epoch, n_outputs)
	predictions = list()
	for row in test:
		prediction = predict(network, row)
		predictions.append(prediction)
	return(predictions)

In [18]:
# Test Backprop on Seeds dataset
seed(1)
# load and prepare data
filename = 'minst.csv'
dataset = load_csv(filename)
for i in range(len(dataset[0])-1):
	str_column_to_float(dataset, i)
# convert class column to integers
str_column_to_int(dataset, len(dataset[0])-1)
# normalize input variables
minmax = dataset_minmax(dataset)
normalize_dataset(dataset, minmax)
# evaluate algorithm
n_folds = 2
l_rate = 0.3
n_epoch = 500
n_hidden = 3
scores = evaluate_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden)
print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores)/float(len(scores))))

KeyboardInterrupt: 

In [13]:
##### Your Code Here #####

## Stretch Goals: 

- Implement Cross Validation model evaluation on your MNIST implementation 
- Research different [Gradient Descent Based Optimizers](https://keras.io/optimizers/)
 - [Siraj Raval the evolution of gradient descent](https://www.youtube.com/watch?v=nhqo0u1a6fw)
- Build a housing price estimation model using a neural network. How does its accuracy compare with the regression models that we fit earlier on in class?